In [14]:
# import requests

# params = {
#     'sv': '',
#     'cuid': '',
#     'os': '',
#     'zid': '',
#     'business': 'return_work',
#     'city_id': '131',
#     'tab_key': 'consume',
#     'module_name': 'tendency',
#     'days': '90',
#     'callback': 'jsonp1673408387540',
# }
# response = requests.get('https://ugc.map.baidu.com/diymap/index_map/main', params=params)
# r=response

In [53]:
import requests

# d=json.loads(r.text.replace('jsonp1673408387540(',"")[:-1])['data']['city_filters']

# # r.text

import time
import json

j=0
d=json.load(open('OneTimeSource\data.json', 'r'))
d=[i for key in d.keys() for i in d[key]]
for i in d[j:]:
    params = {
    'sv': '',
    'cuid': '',
    'os': '',
    'zid': '',
    'business': 'return_work',
    'city_id': i['id'],
    'tab_key': 'consume',
    'module_name': 'tendency',
    'days': '90',
    'callback': 'jsonp1673408387540',
    }
    response = requests.get('https://ugc.map.baidu.com/diymap/index_map/main', params=params)
    r=response
    # time.sleep(3)

    print(j,i['id'],i['name'],r)
    j+=1
    with open('OneTimeSource\BaiduRecovery\%s.json' % i['name'], 'w') as f:
        json.dump(r.text, f)
    


0 131 北京市 <Response [200]>
1 307 保定市 <Response [200]>
2 132 重庆市 <Response [200]>
3 53 长春市 <Response [200]>
4 75 成都市 <Response [200]>
5 158 长沙市 <Response [200]>
6 149 沧州市 <Response [200]>
7 201 潮州市 <Response [200]>
8 348 常州市 <Response [200]>
9 167 大连市 <Response [200]>
10 119 东莞市 <Response [200]>
11 111 大理市 <Response [200]>
12 355 大同市 <Response [200]>
13 372 德州市 <Response [200]>
14 138 佛山市 <Response [200]>
15 300 福州市 <Response [200]>
16 146 贵阳市 <Response [200]>
17 257 广州市 <Response [200]>
18 142 桂林市 <Response [200]>
19 365 赣州市 <Response [200]>
20 48 哈尔滨市 <Response [200]>
21 179 杭州市 <Response [200]>
22 125 海口市 <Response [200]>
23 127 合肥市 <Response [200]>
24 301 惠州市 <Response [200]>
25 321 呼和浩特 <Response [200]>
26 159 衡阳市 <Response [200]>
27 151 邯郸市 <Response [200]>
28 294 湖州市 <Response [200]>
29 208 衡水市 <Response [200]>
30 162 淮安市 <Response [200]>
31 288 济南市 <Response [200]>
32 334 嘉兴市 <Response [200]>
33 302 江门市 <Response [200]>
34 286 济宁市 <Response [200]>
35 333 金华市 <Response [200]>
36 

In [54]:
import pandas as pd
import json,os
dir='./OneTimeSource/BaiduRecovery/'
out=pd.DataFrame()
for f in os.listdir(dir):
    text=open(dir+f,'r',encoding='utf-8').read()[1:-1]
    d=text.replace("jsonp1673408387540(","").replace('''\\"''','''"''')[:-1]
    d=json.loads(d)
    name=d['data']['modules']['tendency']['selected_city']['name']
    points=d['data']['modules']['tendency']['points']
    df= pd.DataFrame([[i['show_name'],i['value']] for i in points],columns=['date','baidu_consumption'])
    df['城市']=name.replace('市','')
    df['date']=pd.to_datetime(df['date'])
    out=pd.concat([out,df])
out.to_hdf(r'Processed_Data/baidu_consumption.h5',key='0112')


In [55]:
df.tail(2)

,date,baidu_consumption,城市
61,2023-01-15,25.26,韶关
62,2023-01-16,24.02,韶关


In [56]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import os
out=pd.read_hdf(r'Processed_Data/baidu_consumption.h5',key='0112')
out

,date,baidu_consumption,城市
0,2022-11-15,28.53,三亚
1,2022-11-16,27.55,三亚
2,2022-11-17,27.96,三亚
3,2022-11-18,28.70,三亚
4,2022-11-19,28.84,三亚
...,...,...,...
58,2023-01-12,18.21,韶关
59,2023-01-13,21.78,韶关
60,2023-01-14,27.73,韶关
61,2023-01-15,25.26,韶关


In [57]:
citydate=pd.read_hdf('Processed_Data/amap.h5',key='noCOVIDdf')
citydate.tail(2)

,城市,date,traffic,WeekDay,yr,dt,mon,traffic_19_same_wd_mean,amapCNY19_last7d_mean,traffic_last7d_mean,traffic_change,traffic_status_old,traffic_status_last7d_mean,CityTier,yrmon,城市date
46027,张家口,2023-01-15,1.38,Sunday,2023,01-15,01,1.405,1.501429,1.501429,0.091381,1.068632,1.000000,四线城市,2023-01,张家口2023-01-15
46028,韶关,2023-01-15,1.35,Sunday,2023,01-15,01,1.405,1.470000,1.442857,0.058700,1.026945,0.981535,四线城市,2023-01,韶关2023-01-15


In [58]:
#set baseline
d=out.copy()
c1=d.date.astype('str')<='2022-11-28'
d_base=d[c1].groupby(by='城市').mean().reset_index().rename(columns={'baidu_consumption':'index_base'})
d=d.merge(d_base,on=['城市'],how='left')

#set 无疫情baseline
c1=d.date.astype('str')<='2022-11-28'
d['城市date']=d['城市']+d['date'].astype('str')
c2=d['城市date'].isin(citydate['城市date'])
d_basenoCOVID=d[c1&c2].groupby(by='城市').mean().reset_index().rename(columns={'baidu_consumption':'index_base_nocovid'})[['城市','index_base_nocovid']]
d=d.merge(d_basenoCOVID,on=['城市'],how='left')

#cal wow
d['index_last7d_mean']=d.groupby('城市')['baidu_consumption'].transform(lambda x: x.rolling(7, 7).mean())
d['index_change_WoW']=(d['index_last7d_mean']/d.groupby('城市')['index_last7d_mean'].transform(lambda x: x.shift(periods=7)))-1


#cal as of 
d['asof2211']=d['index_last7d_mean']/d['index_base']

In [59]:
#恢复情况 as of 无疫情
c1=d['date']>='2022-12-01'
c2=d['index_base_nocovid']>0

t=d[c1 & c2].groupby('date').sum().reset_index()
t['bd']=t['index_last7d_mean']/t['index_base_nocovid']
t[['date','bd']].to_clipboard(index=False)

In [61]:
# 人均收入 vs.bd index WoW

%load_ext autoreload
%autoreload 2
from colored_list import dafeng_city,jidui_city,riqi
import numpy as np
riqi='2023-01-16'

bd=d.copy()
city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')
bd=bd.merge(city_popu,on='城市',how='left')
bd['date']=bd['date'].astype('string') 

bd=bd[bd['date']==riqi].sort_values(by='常住人口',ascending=False)
bd=bd[bd['index_change_WoW']<3].head(70).reset_index()

o=bd[['城市',  '人均可支配收入','index_change_WoW']]

o['size']=np.nan
o['group']='其他'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='本波疫情基本结束'

o=o.sort_values(by='group',ascending=False)

o.to_clipboard(index=False,header=None)
print(riqi,'wow')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2023-01-16 wow


In [71]:
# 人均收入 vs. bd index as of 2022

bd=d.copy()
city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')
bd=bd.merge(city_popu,on='城市',how='left')
bd['date']=bd['date'].astype('string') 

bd=bd[bd['date']==riqi].sort_values(by='常住人口',ascending=False)
bd=bd[bd['asof2211']<3].head(100).reset_index()

o=bd[['城市',  '人均可支配收入','asof2211']]

o['size']=np.nan
o['group']='其他'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='本波疫情基本结束'

o=o.sort_values(by='group',ascending=False)

td=d[d['date']==riqi][['城市','date','asof2211']].reset_index(drop=True).rename(columns={'date':'date','asof2211':'BDcomsu'})
td['date']=pd.to_datetime(td['date'])
td.to_hdf('Processed_Data/city_compare.h5',key='BDcom')

o.to_clipboard(index=False,header=None)
print(riqi,'asof')


2023-01-16 asof


In [64]:
# d1=pd.read_clipboard()
# d1.to_hdf('Processed_Data/tablehead.h5','BDxiaofei')

# d2=pd.pivot_table(d,index=['城市'],columns=['date'],values='baidu_consumption',aggfunc='mean')
# d2.sort_values(by=d2.columns[-1],ascending=False).to_clipboard()

d1=pd.read_hdf('Processed_Data/tablehead.h5','BDxiaofei')
d2=pd.pivot_table(d[d['date'].astype('str')>='2023-01-12'],index='城市',columns='date',values='baidu_consumption',aggfunc='mean').reset_index()
d1.merge(d2,left_on='城市',right_on='城市',how='left').drop(columns=['城市']).to_clipboard(index=False)